In [1]:
#https://www.projectpro.io/article/stock-price-prediction-using-machine-learning-project/571
#https://pypi.org/project/yfinance/
#https://stackoverflow.com/questions/51581521/accuracy-stuck-at-50-keras
#https://towardsdatascience.com/predicting-stock-prices-using-a-keras-lstm-model-4225457f0233
#https://machinelearningknowledge.ai/keras-lstm-layer-explained-for-beginners-with-example/
#https://www.kaggle.com/code/nizado/keras-for-stocks-prediction
!pip install yfinance
!pip install requests-cache
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install tensorflow
!pip install keras
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 433 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 494 kB/s 
     |████████████████████████████████| 140 kB 8.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in ind

In [2]:
import requests_cache
import yfinance as yf
import pandas_ta as ta

selection = 'AAPL AMD GOOGL NVDA'
data = yf.Ticker('AAPL').history(period='5y')[['Close', 'Open', 'High', 'Volume', 'Low']]

In [3]:
'''close='Close'
name='aapl'
data.ta.macd(close=close, fast=12, slow=26, signal=9, append=True)
data.ta.rsi(close=close, length=14, append=True)
data.ta.bias(close=close, append=True)
data.ta.adx(close=close, append=True)
data.ta.stdev(close=close, append=True)

data.to_csv(f"/content/data/{name}")'''


'close=\'Close\'\nname=\'aapl\'\ndata.ta.macd(close=close, fast=12, slow=26, signal=9, append=True)\ndata.ta.rsi(close=close, length=14, append=True)\ndata.ta.bias(close=close, append=True)\ndata.ta.adx(close=close, append=True)\ndata.ta.stdev(close=close, append=True)\n\ndata.to_csv(f"/content/data/{name}")'

In [39]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)

data = pd.read_csv('/content/sample_data/nvda')
data_inputs = data.iloc[:, 1:14].values
data_class = data.iloc[:, 15:16].values
print(data_class)

[[3]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [40]:
#Classes 'B_Small', 'B_large', or 'Sell' 
np.asarray(data_inputs).astype('float32')
np.asarray(data_class).astype('float32')

array([[3.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

x_train, x_test, y_train, y_test = train_test_split(data_inputs, data_class, train_size=0.75)

In [ ]:
#Standardization
sc2 = StandardScaler()
sc2.fit(x_train)
sc2.transform(x_train)
sc2.transform(x_test)

In [48]:
#Normalization
sc = MinMaxScaler()
sc.fit(x_train)
sc.transform(x_train)
sc.transform(x_test)

array([[0.087, 0.082, 0.082, ..., 0.345, 0.402, 0.516],
       [0.121, 0.122, 0.116, ..., 0.245, 0.43 , 0.521],
       [0.059, 0.056, 0.055, ..., 0.   , 0.371, 0.452],
       ...,
       [0.085, 0.086, 0.083, ..., 0.559, 0.217, 0.776],
       [0.332, 0.32 , 0.317, ..., 0.502, 0.327, 0.596],
       [0.511, 0.513, 0.497, ..., 0.658, 0.375, 0.648]])

In [59]:
import keras as k
from keras.models import *
from keras.layers import *

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
x_test.shape

(315, 1, 13)

In [75]:
seqModel = Sequential()
#seqModel.add(Reshape((13,1)))
#print(seqModel.output_shape)
#print(seqModel.input_shape)
#seqModel.add(Normalization(axis=None).adapt(data_input))

In [76]:

seqModel.add(InputLayer((1,13)))
seqModel.add(LSTM(units=48, return_sequences=True, dropout=0.2))
seqModel.add(LSTM(units=64, return_sequences=True, dropout=0.2))
seqModel.add(LSTM(units=96, return_sequences=True, dropout=0.2))
seqModel.add(LSTM(units=96, dropout=0.2))
seqModel.add(Dense(units = 2, activation='sigmoid'))


In [81]:
seqModel.build()
seqModel.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_32 (LSTM)              (None, 1, 48)             11904     
                                                                 
 lstm_33 (LSTM)              (None, 1, 64)             28928     
                                                                 
 lstm_34 (LSTM)              (None, 1, 96)             61824     
                                                                 
 lstm_35 (LSTM)              (None, 96)                74112     
                                                                 
 dense_7 (Dense)             (None, 2)                 194       
                                                                 
Total params: 176,962
Trainable params: 176,962
Non-trainable params: 0
_________________________________________________________________


In [82]:
from keras.losses import *
from keras.optimizers import *
from keras.metrics import *

lossFunc = SparseCategoricalCrossentropy()
batchSize = 1 #from 64 -> 32 for test
epochs = 10 # from 100 -> 10 for test
metrics=['mean_squared_error']
seqModel.compile(optimizer='nadam', loss=lossFunc, metrics=metrics)
seqModel.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = epochs, batch_size = batchSize, validation_batch_size=batchSize)

Epoch 1/10


ValueError: ignored

In [34]:
print(x_train.shape) 
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(944, 1, 13)
(315, 1, 13)
(944, 1)
(315, 1)


In [ ]:
trainMod = seqModel.evaluate(x_train, y_train)
testMod = seqModel.evaluate(x_test, y_test)
print(trainMod)
print(testMod)

trainPred = seqModel.predict((x_train.reshape(-1,4)))
print(trainPred)
testPred = seqModel.predict((x_test.reshape(-1,4)))
print(testPred)

In [ ]:
#jaycode
seqModel.save('sp.h5')
#@title jaycode
seqModel = load_model('sp.h5')

predictions = seqModel.predict(x_test)
predictions = sc.inverse_transform(predictions)
y_test_scaled = sc.inverse_transform(y_test.reshape(-1, 1))

fig, ax = plt.subplots(figsize=(16,8))
ax.set_facecolor('#000041')
ax.plot(y_test_scaled, color='red', label='Actual price')
plt.plot(predictions, color='cyan', label='Predicted price')
plt.legend()

In [ ]:
trainMod = seqModel.evaluate(x_train, y_train)
testMod = seqModel.evaluate(x_test, y_test)
print(trainMod)
print(testMod)

trainPred = seqModel.predict((x_train.reshape(-1,4)))
print(trainPred)
testPred = seqModel.predict((x_test.reshape(-1,4)))
print(testPred)

